In [1]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

In [2]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [16]:
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 34.0588,
	"longitude": -118.4439,
	"start_date": "2024-04-01",
	"end_date": "2024-06-30",
	"hourly": ["temperature_2m", "precipitation", "relative_humidity_2m", "cloud_cover"]
}
responses = openmeteo.weather_api(url, params=params)

In [17]:
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 34.05975341796875°N -118.46250915527344°E
Elevation 103.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s


In [18]:
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(3).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["cloud_cover"] = hourly_cloud_cover

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

                          date  temperature_2m  precipitation  \
0    2024-04-01 00:00:00+00:00       14.844999            0.1   
1    2024-04-01 01:00:00+00:00       12.295000            0.2   
2    2024-04-01 02:00:00+00:00       11.594999            0.3   
3    2024-04-01 03:00:00+00:00       10.945000            0.0   
4    2024-04-01 04:00:00+00:00        9.995000            0.0   
...                        ...             ...            ...   
2179 2024-06-30 19:00:00+00:00       28.195000            0.0   
2180 2024-06-30 20:00:00+00:00       28.844999            0.0   
2181 2024-06-30 21:00:00+00:00       28.994999            0.0   
2182 2024-06-30 22:00:00+00:00       29.494999            0.0   
2183 2024-06-30 23:00:00+00:00       29.295000            0.0   

      relative_humidity_2m  cloud_cover  
0                71.525261         65.0  
1                78.935852         69.0  
2                84.068306         65.0  
3                87.766647         38.0  
4        

In [19]:
# Change title of csv based on retrieved data
hourly_dataframe.to_csv('spring-2024-weather.csv', index = False)